In [1]:
%load_ext autoreload
%autoreload 2
!hostname
!pwd
import os, sys
print(sys.executable)
# os.environ['CUDA_VISIBLE_DEVICES'] = "7"
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = "false"
# sys.path.append(os.path.abspath(".."))

slurm0-gpu1nodeset-0
/home/akarsh_sakana_ai/spaghetti/src
/home/akarsh_sakana_ai/.conda/envs/spaghetti/bin/python


In [2]:
import os, sys, glob, pickle, copy, time
from functools import partial

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = (15, 5)
import seaborn as sns
sns.set_theme()

from tqdm.auto import tqdm
from einops import rearrange, reduce, repeat
from einop import einop

import jax
import jax.numpy as jnp
from jax.random import split

import flax
import flax.linen as nn
from flax.training.train_state import TrainState

import optax

In [28]:
from clip import CLIPOriginal, CLIP

In [29]:
clip1, clip2 = CLIPOriginal(), CLIP()

In [30]:
img_cat = plt.imread("../cat.jpg")/255.
img_dog = plt.imread("../dog.jpg")/255.
img_cat = jax.image.resize(img_cat, (224, 224, 3), method='bilinear')
img_dog = jax.image.resize(img_dog, (224, 224, 3), method='bilinear')
imgs = jnp.stack([img_cat, img_dog])

In [ ]:
for clip in [clip1, clip2]:
    z_txt = clip.embed_txt(['a photo of a cat', 'a photo of a dog', 'a photo of a human'])
    z_img = jax.vmap(clip.embed_img)(imgs)
    scores = z_img @ z_txt.T
    # print(scores)

    probs = jax.nn.softmax(scores*100, axis=-1)
    print(probs)

[[0.9828562  0.01041402 0.00672975]
 [0.00148572 0.9880585  0.01045577]]
[[0.98288965 0.01039237 0.00671803]
 [0.00148728 0.98806936 0.01044337]]
